In [ ]:
!pip install nltk
!pip install pickle5
!pip install tensorflow

In [ ]:
import json
import pickle
import numpy as np
import nltk
import random
import re
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
def guardarjson(datos, filename):
  archivo=open(filename,"w")
  json.dump(datos,archivo,indent=4)

dic_palabras={"intents":
            [{"tag":"saludos",
              "patterns":['hola','consulta','saludos','buenas','una consulta','una pregunta' ],
              "responses":['Hola soy Luna, tu asistente virtual. ¿En qué puedo ayudarte?']
            },
             {"tag":"ubicacion",
              "patterns":['ubicados','direccion','donde','ubicacion','encontrarlo'],
              "responses":['Estamos ubicados en Calle5ta. Rio Abajo, Panamá, República de Panamá, Galera Empresas Carbone']
             },
             {"tag":"horario",
              "patterns":['horario', 'cierre','abierto'],
              "responses":['Nuestro horario es: \nLunes a Jueves de 8:00 a.m. a 6:00 p.m.\nViernes de 8:00 a.m. a 5:00 p.m.\nSábados y Domingos CERRADOS|']
              },
             {"tag":"contacto",
              "patterns":['contacto','numero','correo','a donde puedo comunicarme con ustedes','comunicacion'],
              "responses":['Tel.(+507)391-6309/391-6313\nContacto: ricardo@carbone.com.pa']
              },
              {"tag":"despedidas",
               "patterns":['gracias','te lo agradezco','thanks'],
               "responses":['Estamos a la orden']
               },
             {"tag":"beneficios",
              "patterns":['que beneficios tiene adquirir este producto','que beneficios tiene adquirir este producto','beneficios','utilidad','mejoria'],
              "responses":['Podras encontrar ofertas en mas de 184 productos de nuestra tienda, adquirir tarjetas de regalo y mas.']
              },
             {"tag":"necesidad",
              "patterns":['Puedo recibir informacion regular al correo con todo lo nuevo de la web','informacion','recibir'],
              "responses":['Al suscribirte a nuestra pagina web te llegara todo lo nuevo de nuestra tienda.']
              },
             {"tag":"pedidos",
              "patterns":['como puedo cancelar un pedido','cancelacion','reclamo','garantia'],
              "responses":["Debe llamar al departamento de atencion al cliente, a nuestro telefono o en enviar un e-mail para realizar dicha operacion."]
             },
             {"tag":"metodo_pago",
              "patterns":['que metodos de pago aceptan','como se puede pagar','pagar','pago','aceptan tarjeta','efectivo'],
              "responses":["Existen 2 metodos de pagos disponibles: En efectivo y pago en linea"]
              },
             {"tag":"cotizacion",
              "patterns":['listo','demora','en cuanto tiempo tendre listo mi producto'],
              "responses":["El tiempo de entrega depende del destino a donde es enviado el producto."]
              },
             {"tag":"norespuesta",
              "patterns":[""],
              "responses":["no se detecto respuesta"]
              }
              ]
            }
guardarjson(dic_palabras, 'intents.json')
dic_palabras

In [ ]:
import nltk
nltk.download('punkt')
bolsadepalabras = [] 
documento = []     
clase = []             
for intent in dic_palabras['intents']:
    clase.append(intent['tag'])
    for pattern in intent['patterns']:       
        result = nltk.word_tokenize(pattern)        
        bolsadepalabras.extend(result)
        documento.append((result, intent['tag']))
print(bolsadepalabras)     
print(clase)
for elemento in documento:
    print('\n')
    print(elemento)

In [ ]:

l1 = 'buen dias'
l2 = 'buen tardes'
l3 = 'bienvenidos'

bagofwords= []                 
bagofwords.extend(l1.split())  
bagofwords.extend(l2.split())  
bagofwords.extend(l3.split())  
print(bagofwords)
print("Bolsa de palabras")
print(bagofwords)
print(len(bagofwords))
l1binario = [1, 1, 0 ,0 ,0] 
l2binario = [1, 0, 0, 0, 1] 
l3binario = [0, 0, 0, 1, 0] 
print('\n')
print('l1 como string', '\t'*2,l1)
print('l1 como binario', '\t'*1, l1binario)

In [ ]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')
proh_palbr = ["?","¿","!","!",".","|",",",":",";","-","_"]                  
bolsadepalabras2 = []                                
for w in bolsadepalabras:                            
    if w not in proh_palbr:
        wprocesada = w.lower()                               
        wprocesada = stemmer.stem(wprocesada)       
        bolsadepalabras2.append(wprocesada)          
print("bolsadepalabras2:", bolsadepalabras2)

In [ ]:
def limpieza(words, proh_palbr):
    words = [stemmer.stem(w.lower()) for w in words if w not in proh_palbr]
    return words
bolsadepalabras = limpieza(bolsadepalabras, proh_palbr)
pickle.dump(bolsadepalabras,open("bolsadepalabras.pkl","wb")) 
pickle.dump(clase,open("classes.pkl","wb"))        
entrenar = []

for doc in documento:    
    interaccion = doc[0]           
    interaccion = limpieza(interaccion, proh_palbr) 
    entradacodificada = [] 
    for palabra in bolsadepalabras:
        if palabra in interaccion:
            entradacodificada.append(1)
        else:
            entradacodificada.append(0)    
    salidacodificada = [0]*len(clase)
    indice = clase.index(doc[1])
    salidacodificada[indice] = 1
    entrenar.append([entradacodificada, salidacodificada])
    
entrenar = np.array(entrenar, dtype=list)
entrenarx = list(entrenar[:,0])
entrenary = list(entrenar[:,1])
print(entrenarx[0])
print(len(entrenary[0]))

In [ ]:
model = Sequential()
model.add(Dense(128,input_shape=(len(entrenarx[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(entrenary[0]),activation='softmax'))

In [ ]:
sgd = SGD(learning_rate=0.01, momentum=0.9,nesterov=True)
model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=["accuracy"])
hist=model.fit(np.array(entrenarx),np.array(entrenary),epochs=300,batch_size=5,verbose=True)
model.save("chatbot_model.h5",hist)
print("MODELO CREADO")

In [ ]:
from tensorflow.keras.models import load_model
import json
import pickle
import numpy as np
import nltk
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

proh_palbr = ["?","¿","!","!",".","|",",",":",";","-","_"] 
model=load_model("chatbot_model.h5")
dic_palabras=json.loads(open("intents.json").read())
bolsadepalabras=pickle.load(open("bolsadepalabras.pkl","rb"))
clase=pickle.load(open("classes.pkl","rb"))
print(len(bolsadepalabras))

In [ ]:
def cleanEntrada(impr_usuario):
    impr_usuario = nltk.word_tokenize(impr_usuario)
    impr_usuario= [stemmer.stem(w.lower()) for w in impr_usuario if w not in proh_palbr]
    return impr_usuario

def conVector(impr_usuario, bolsadepalabras):
    
    impr_usuario = cleanEntrada(impr_usuario)
    
    vectorentrada = [0]*len(bolsadepalabras)       
    for palabra in impr_usuario:              # loop sobre la entrada del usuario
        
        if palabra in bolsadepalabras:          # verificación si la palabra esta dentro de la bolsa de palabras.
            
            indice = bolsadepalabras.index(palabra)    
            vectorentrada[indice] = 1                  
            
    vectorentrada = np.array(vectorentrada)            #  conversión a un arreglo numpy
    return vectorentrada

entradausuario ="buenos dias"

In [ ]:
def gettag(vectorentrada, LIMITE = 0):
    vectorsalida = model.predict(np.array([vectorentrada]))[0]

    vectorsalida = [[i,r] for i,r in enumerate(vectorsalida) if r > LIMITE]    

    vectorsalida.sort(key=lambda x: x[1], reverse=True) 
    print(vectorsalida)
    
    listEtiquetas = []    
    for r in vectorsalida:   
        listEtiquetas.append({"intent": clase[r[0]], "probability": str(r[1])})   
    return listEtiquetas

listEtiquetas = gettag(vectorentrada, LIMITE = 0.1)
listEtiquetas

In [ ]:
def getResponse (listEtiquetas,dic_palabras):
etiqueta=listEtiquetas[0]['intent']
  listadediccionarios=dic_palabras['intents']
for dic_palabras in listadediccionarios:

    if etiqueta==dic_palabras['tag']:
    listaDeRespuestas=dic_palabras['responses']
      respuesta=random.choice(listaDeRespuestas)
    break
return respuesta
respuesta=getResponse(listEtiquetas, dic_palabras)
respuesta

In [ ]:
def chatbotResponde(impr_usuario):
vectorentrada=conVector(impr_usuario,bolsadepalabras)
  listEtiquetas=gettag(vectorentrada,LIMITE=0)
respuesta=getResponse(listEtiquetas,dic_palabras)
  return respuesta

In [ ]:
def menu():
      print("Bienvenido cliente ")
  print("1)Cotizacion o presupuesto")
      print("2)Gestionar citas")
  print("3)Contactar con un agente")
      print("4)Otro")
  print("5)Salir")
#que escoga la opcion y si es otro, lo mandamos al asistente virtual... 

In [ ]:
impr_usuario=''

while impr_usuario!='exit':
impr_usuario=input()
  respuesta= chatbotResponde(impr_usuario)
print(respuesta)
